# Gurobi develop app ML TIPS - PROCESS - TANKS

In [ ]:
import pickle
import pandas as pd
import numpy as np
import json
import gurobipy_pandas as gppd
from gurobi_ml import add_predictor_constr
import gurobipy as gp
import plotly.graph_objects as go

from optimization_engine import optimization_engine

## I) Read env variables

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv # package used in jupyter notebook to read the variables in file .env

""" get env variable from .env """
load_dotenv(find_dotenv())

""" Read env variables and save it as python variable """
WLSACCESSID = os.environ.get("WLSACCESSID", "")
WLSSECRET = os.environ.get("WLSSECRET", "")
LICENSEID = int(os.environ.get("LICENSEID", ""))
PROJECT_GCP = os.environ.get("PROJECT_GCP", "")

## II) LOAD LICENCE GUROBI - using env variables

In [ ]:
import gurobipy as gp
params = {
"WLSACCESSID": WLSACCESSID,
"WLSSECRET": WLSSECRET,
"LICENSEID": LICENSEID
}
env = gp.Env(params=params)

## III) FORM STREAMLIT INGRESS DATA

### LOAD DATA - INPUT VALUES - USER

### I) Initial values

In [ ]:
####### INITIAL VALUES
input_X1_initial = 50
input_O1_initial = 50
input_O2_initial = 50
input_O3_initial = 50

input_O4_initial = 200

input_X2_initial = 5
input_O5_initial = 5
input_O6_initial = 5

input_X3_initial = 5
input_O7_initial = 4

input_TL1_initial = 500
input_TL2_initial = 500
input_TL3_initial = 500

### II) Lower bound, upper bound and rate change
In this example, only primary, target, secondary variables and tank level have defined lower bound, upper bound and rate change. So, the observed variable hasn't defined that

In [ ]:
# X1
input_X1_lower_bound = 0
input_X1_upper_bound = 1000
input_X1_rate_change = 100
		
# Y1
input_Y1_lower_bound = 0
input_Y1_upper_bound = 400
input_Y1_rate_change = 100
		
# Z1
input_Z1_lower_bound = 0
input_Z1_upper_bound = 1000
input_Z1_rate_change = 100
		
# X2
input_X2_lower_bound = 0
input_X2_upper_bound = 1000
input_X2_rate_change = 100

# Y2
input_Y2_lower_bound = 0
input_Y2_upper_bound = 500
input_Y2_rate_change = 100

# Y3
input_Y3_lower_bound = 0
input_Y3_upper_bound = 450
input_Y3_rate_change = 100

# X3
input_X3_lower_bound = 0
input_X3_upper_bound = 1500
input_X3_rate_change = 100

# TL1
input_TL1_lower_bound = 100
input_TL1_upper_bound = 20000
input_TL1_rate_change = 100

# TL2
input_TL2_lower_bound = 100
input_TL2_upper_bound = 20000
input_TL2_rate_change = 100

# TL3
input_TL3_lower_bound = 100
input_TL3_upper_bound = 20000
input_TL3_rate_change = 100

## IV) IF THE USER INGRESS THE INFORMATION (SUBMIT BUTTON IN FORM)
### GENERATE DATAFRAMES AND RUN OPTIMIZATION ENGINE

### I) Initial values
The initial values by default is a excel file read as dataframe. So, for this example the dataframe is recreated

In [ ]:
#################### read original initial values - default excel file
# paths and file names
path_folder_config_optimization = f'config/optimization_engine/config_optimization/'
file_initvalues = 'InitialValues.xlsx'
path_initvalues = path_folder_config_optimization + file_initvalues

# read file
config_initvalues = pd.read_excel(path_initvalues)


#################### update values with the input values
config_initvalues.loc[config_initvalues['feature_name'] == 'X1', 'init_values'] = input_X1_initial
config_initvalues.loc[config_initvalues['feature_name'] == 'O1', 'init_values'] = input_O1_initial
config_initvalues.loc[config_initvalues['feature_name'] == 'O2', 'init_values'] = input_O2_initial
config_initvalues.loc[config_initvalues['feature_name'] == 'O3', 'init_values'] = input_O3_initial

config_initvalues.loc[config_initvalues['feature_name'] == 'O4', 'init_values'] = input_O4_initial

config_initvalues.loc[config_initvalues['feature_name'] == 'X2', 'init_values'] = input_X2_initial
config_initvalues.loc[config_initvalues['feature_name'] == 'O5', 'init_values'] = input_O5_initial
config_initvalues.loc[config_initvalues['feature_name'] == 'O6', 'init_values'] = input_O6_initial

config_initvalues.loc[config_initvalues['feature_name'] == 'X3', 'init_values'] = input_X3_initial
config_initvalues.loc[config_initvalues['feature_name'] == 'O7', 'init_values'] = input_O7_initial

config_initvalues.loc[config_initvalues['feature_name'] == 'TL1', 'init_values'] = input_TL1_initial
config_initvalues.loc[config_initvalues['feature_name'] == 'TL2', 'init_values'] = input_TL2_initial
config_initvalues.loc[config_initvalues['feature_name'] == 'TL3', 'init_values'] = input_TL3_initial

### II) Lower bound, upper bound and rate change
The lower, upper and rate change by default is a excel file read as dataframe. So, for this example the dataframe is recreated

In [ ]:
#################### read original initial values - default excel file
# paths and file names
path_folder_config_optimization = f'config/optimization_engine/config_optimization/'
file_allvariables = 'AllVariables.xlsx'
path_allvariables = path_folder_config_optimization + file_allvariables

# read file
config_allvariables = pd.read_excel(path_allvariables)


#################### update values with the input values
# X1
config_allvariables.loc[config_initvalues['feature_name'] == 'X1', 'lower'] = input_X1_lower_bound
config_allvariables.loc[config_initvalues['feature_name'] == 'X1', 'upper'] = input_X1_upper_bound
config_allvariables.loc[config_initvalues['feature_name'] == 'X1', 'rate_change'] = input_X1_rate_change
		
# Y1
config_allvariables.loc[config_initvalues['feature_name'] == 'Y1', 'lower'] = input_Y1_lower_bound
config_allvariables.loc[config_initvalues['feature_name'] == 'Y1', 'upper'] = input_Y1_upper_bound
config_allvariables.loc[config_initvalues['feature_name'] == 'Y1', 'rate_change'] = input_Y1_rate_change
		
# Z1
config_allvariables.loc[config_initvalues['feature_name'] == 'Z1', 'lower'] = input_Z1_lower_bound
config_allvariables.loc[config_initvalues['feature_name'] == 'Z1', 'upper'] = input_Z1_upper_bound
config_allvariables.loc[config_initvalues['feature_name'] == 'Z1', 'rate_change'] = input_Z1_rate_change
		
# X2
config_allvariables.loc[config_initvalues['feature_name'] == 'X2', 'lower'] = input_X2_lower_bound
config_allvariables.loc[config_initvalues['feature_name'] == 'X2', 'upper'] = input_X2_upper_bound
config_allvariables.loc[config_initvalues['feature_name'] == 'X2', 'rate_change'] = input_X2_rate_change

# Y2
config_allvariables.loc[config_initvalues['feature_name'] == 'Y2', 'lower'] = input_Y2_lower_bound
config_allvariables.loc[config_initvalues['feature_name'] == 'Y2', 'upper'] = input_Y2_upper_bound
config_allvariables.loc[config_initvalues['feature_name'] == 'Y2', 'rate_change'] = input_Y2_rate_change

# Y3
config_allvariables.loc[config_initvalues['feature_name'] == 'Y3', 'lower'] = input_Y3_lower_bound
config_allvariables.loc[config_initvalues['feature_name'] == 'Y3', 'upper'] = input_Y3_upper_bound
config_allvariables.loc[config_initvalues['feature_name'] == 'Y3', 'rate_change'] = input_Y3_rate_change

# X3
config_allvariables.loc[config_initvalues['feature_name'] == 'X3', 'lower'] = input_X3_lower_bound
config_allvariables.loc[config_initvalues['feature_name'] == 'X3', 'upper'] = input_X3_upper_bound
config_allvariables.loc[config_initvalues['feature_name'] == 'X3', 'rate_change'] = input_X3_rate_change

# TL1
config_allvariables.loc[config_initvalues['feature_name'] == 'TL1', 'lower'] = input_TL1_lower_bound
config_allvariables.loc[config_initvalues['feature_name'] == 'TL1', 'upper'] = input_TL1_upper_bound
config_allvariables.loc[config_initvalues['feature_name'] == 'TL1', 'rate_change'] = input_TL1_rate_change

# TL2
config_allvariables.loc[config_initvalues['feature_name'] == 'TL2', 'lower'] = input_TL2_lower_bound
config_allvariables.loc[config_initvalues['feature_name'] == 'TL2', 'upper'] = input_TL2_upper_bound
config_allvariables.loc[config_initvalues['feature_name'] == 'TL2', 'rate_change'] = input_TL2_rate_change

# TL3
config_allvariables.loc[config_initvalues['feature_name'] == 'TL3', 'lower'] = input_TL3_lower_bound
config_allvariables.loc[config_initvalues['feature_name'] == 'TL3', 'upper'] = input_TL3_upper_bound
config_allvariables.loc[config_initvalues['feature_name'] == 'TL3', 'rate_change'] = input_TL3_rate_change

In [ ]:
config_allvariables

## V) LOAD CONFIGURATION FILES FOR OPTIMIZER

#### 1.1. IndexTime file

In [ ]:
#################### define set ####################

## paths and files names
path_folder_config_optimization = f'config/optimization_engine/config_optimization/'
file_indextime = 'IndexTime.xlsx'
path_indextime = path_folder_config_optimization + file_indextime

# read file
indextime = pd.read_excel(path_indextime)

# set index
index_set_time = pd.Index(indextime['IndexTime'].values)

#### 1.2 Decision variables

In [ ]:
# CREATED USING USER INPUTS

#### 1.3 Initial Values

In [ ]:
# CREATED USING USER INPUTS

#### 1.4 Define models to load

In [ ]:
#################### define initial values ####################

# paths and file names
path_folder_config_optimization = f'config/optimization_engine/config_optimization/'
file_modelsml = 'ModelsML.xlsx'
path_modelsml = path_folder_config_optimization + file_modelsml

# read file
config_modelsml = pd.read_excel(path_modelsml)

#### 1.5 Map tanks

In [ ]:
#################### define map input outputs each tank ####################

# paths and file names
path_folder_config_optimization = f'config/optimization_engine/config_optimization/'
file_maptanks = 'MapTanks.xlsx'
path_maptanks = path_folder_config_optimization + file_maptanks

# read file
config_maptanks = pd.read_excel(path_maptanks)

#### 1.6 Map process Machine learning models features and target

In [ ]:
#################### define map features and targer of each model for each process ####################

# paths and file names
path_folder_config_optimization = f'config/optimization_engine/config_optimization/'
file_mapprocess_mlmodels = 'MapProcessMLmodels.xlsx'
path_mapprocess_mlmodels = path_folder_config_optimization + file_mapprocess_mlmodels

# read filemapprocess_mlmodels
config_mapprocess_mlmodels = pd.read_excel(path_mapprocess_mlmodels)

## VI) RUN OPTIMIZATION

#### i) Define parameters used to relax constraints (if it is necessary)

In [ ]:
# define list of decision vars that will have change in its rate change
list_tags_to_relax_constraints = ['TL1', 'TL2', 'TL3']

# define factor (percent = 0.1) to change the rate change of the decision vars. 
# In this example all the decision var masked will change its value in the same factor
param_factor_relax_constraints = 0.1
factor_relax_constraints = 1 + param_factor_relax_constraints

#### ii) run optimization

In [ ]:
######## do while. do define status solver // while status solver != 2 relaxing contrainsts (rate of change decision variables) until get a solution
# initialize status
status_solver = 0

# initialize count iterations of relaxing constraints
index_count_while = 0
max_interations_relaxing_constraints = 40

# initiliaze values of constraints to relax
to_solver_config_allvariables = config_allvariables.copy()



while (status_solver != 2) & (index_count_while <=max_interations_relaxing_constraints):
    print('\n\n\n index while: ', index_count_while)

    ###### run optimization
    model_opt, status_solver, decision_var = optimization_engine(index_set_time,
                                    config_allvariables,
                                    config_initvalues,
                                    config_modelsml,
                                    config_maptanks,
                                    config_mapprocess_mlmodels,
                                    params)

    ##### get status solver - if solver get a solution - break the while
    if status_solver == 2:
        break
    
    # ###### relaxing constraints. if the solver return a value this values was delete, else the relaxing constraints are used in the while to get a solution
    # multiply rate change by factor of selected features
    mask = config_allvariables['feature_name'].isin(list_tags_to_relax_constraints)
    config_allvariables.loc[mask, 'rate_change'] = config_allvariables[mask]['rate_change'] * factor_relax_constraints

    # increse while
    index_count_while += 1

In [ ]:
# check status solver
print('status solver: ' , status_solver)
if status_solver != 2:
    print('Infeasible solution')

In [ ]:
##### REVISAR QUÉ VALORES DE RATE CHANGE QUEDARON DE LAS VARIABLES
mask = config_allvariables['feature_name'].isin(list_tags_to_relax_constraints)
config_allvariables[mask]

## VII) SHOW FRONT IF THE SOLVER WORKS

In [ ]:
######################## get optimal values and save in a dataframe
# create a dataframe with set as index
solution = pd.DataFrame(index = index_set_time)

# save optimal values - features of models (only the features)
solution["var_X1"] = decision_var['X1'].gppd.X
solution["var_O1"] = decision_var['O1'].gppd.X
solution["var_O2"] = decision_var['O2'].gppd.X
solution["var_O3"] = decision_var['O3'].gppd.X
solution["var_Y1"] = decision_var['Y1'].gppd.X
solution["var_O4"] = decision_var['O4'].gppd.X
solution["var_Z1"] = decision_var['Z1'].gppd.X
solution["var_X2"] = decision_var['X2'].gppd.X
solution["var_O5"] = decision_var['O5'].gppd.X
solution["var_O6"] = decision_var['O6'].gppd.X
solution["var_Y2"] = decision_var['Y2'].gppd.X
solution["var_Y3"] = decision_var['Y3'].gppd.X
solution["var_X3"] = decision_var['X3'].gppd.X
solution["var_O7"] = decision_var['O7'].gppd.X
solution["var_TL1"] = decision_var['TL1'].gppd.X
solution["var_TL2"] = decision_var['TL2'].gppd.X
solution["var_TL3"] = decision_var['TL3'].gppd.X

# get value objetive function
opt_objetive_function = model_opt.ObjVal

In [ ]:
# show value objetive function
opt_objetive_function

In [ ]:
# show value decision variables
solution.T